In [17]:
import pandas as pd
import numpy as np
import datetime as dt

In [18]:
df_match= pd.read_csv("matches_info_cleaned.csv", sep=',') 
df_player = pd.read_csv('players_info_clustezition.csv', sep=',')

In [19]:
df_player = df_player.iloc[0:100]
df_player['name_player'] = df_player['player_name']
df_player.drop(columns=['player_name'], inplace=True)

In [20]:
df = df_match.merge(df_player, left_on='adv_name', right_on='name_player')

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26111 entries, 0 to 26110
Data columns (total 41 columns):
surface               26111 non-null object
tourney_level         26111 non-null object
tourney_date          26111 non-null int64
player_id_x           26111 non-null int64
player_name           26111 non-null object
player_hand           26111 non-null object
adv_id                26111 non-null int64
adv_name              26111 non-null object
adv_hand              26111 non-null object
round                 26111 non-null object
diff_rank             26111 non-null float64
diff_rank_points      26111 non-null float64
diff_age              26111 non-null float64
class                 26111 non-null int64
player_id_y           26111 non-null int64
age                   26111 non-null int64
country               26111 non-null object
favorite_hand         26111 non-null object
backhand              26111 non-null object
turned_pro            26111 non-null int64
prize_money    

In [22]:
df_filter = df.groupby('name_player').size().reset_index(name='counts')
df_filter = df_filter[df_filter['counts'] >= 100]

df = df.merge(df_filter, on='name_player')

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24687 entries, 0 to 24686
Data columns (total 42 columns):
surface               24687 non-null object
tourney_level         24687 non-null object
tourney_date          24687 non-null int64
player_id_x           24687 non-null int64
player_name           24687 non-null object
player_hand           24687 non-null object
adv_id                24687 non-null int64
adv_name              24687 non-null object
adv_hand              24687 non-null object
round                 24687 non-null object
diff_rank             24687 non-null float64
diff_rank_points      24687 non-null float64
diff_age              24687 non-null float64
class                 24687 non-null int64
player_id_y           24687 non-null int64
age                   24687 non-null int64
country               24687 non-null object
favorite_hand         24687 non-null object
backhand              24687 non-null object
turned_pro            24687 non-null int64
prize_money    

In [24]:
from datetime import date


def int2date(argdate: int) -> date:
   
    year = int(argdate / 10000)
    month = int((argdate % 10000) / 100)
    day = int(argdate % 100)

    return date(year, month, day)

In [25]:
df['date'] = df['tourney_date'].apply(lambda x: int2date(x)).apply(pd.to_datetime)

In [26]:
df.drop(columns=['tourney_date', 'player_id_x', 'player_id_y', 'adv_id', 'age', 'favorite_hand', 
                 'prize_money', 'turned_pro', 'country', 
                 'adv_name', 'player_name'], inplace=True)

In [27]:
df = df[~df['surface'].isin(['Carpet', 'None'])]

In [28]:
df_train = df[ df['date'] < dt.date(2018,1,1) ]
df_test = df[ df['date'] >= dt.date(2018,1,1) ]

df_train_clus =  pd.get_dummies(df_train)
df_test_clus =  pd.get_dummies(df_test)

df_test_clus = df_test_clus[df_test_clus['tourney_level_G'] == 1]

/Users/suzana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  """Entry point for launching an IPython kernel.
/Users/suzana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  


In [218]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 100, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [219]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

clf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [220]:
clf_random.fit(df_train_clus.drop(columns=['class', 'date']), df_train_clus['class'])

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 12.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 25.1min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [222]:
clf_random.best_params_

{'n_estimators': 400,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': True}

In [30]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=400,  min_samples_split=2, min_samples_leaf = 4, max_features = 'sqrt',
                            max_depth = 10, bootstrap= True)
clf.fit(df_train_clus.drop(columns=['class', 'date']), df_train_clus['class'])
pred = clf.predict(df_test_clus.drop(columns=['class', 'date']))

In [31]:
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(df_test_clus['class'], pred)

0.7122140684043943

In [14]:
from sklearn.svm import SVC

clf = SVC(gamma='auto')
clf.fit(df_train_clus.drop(columns=['class', 'date']), df_train_clus['class']) 
pred = clf.predict(df_test_clus.drop(columns=['class', 'date']))